In [ ]:
!pip install pgmpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.8 MB/s eta 0:00:00


In [ ]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Step 1: Define the structure of the Bayesian Network
model = BayesianNetwork([
    ('Season', 'Cold'),  # Season influences Cold
    ('Cold', 'Fever'),   # Cold influences Fever
    ('Cold', 'RunnyNose')  # Cold influences Runny Nose
])

# Step 2: Define Conditional Probability Distributions (CPDs)
# P(Season)
cpd_season = TabularCPD(variable='Season', variable_card=2, values=[[0.7], [0.3]])
# P(Cold | Season)
cpd_cold = TabularCPD(variable='Cold', variable_card=2,
                      values=[[0.8, 0.2],  # P(Cold=False | Season)
                              [0.2, 0.8]], # P(Cold=True | Season)
                      evidence=['Season'], evidence_card=[2])
# P(Fever | Cold)
cpd_fever = TabularCPD(variable='Fever', variable_card=2,
                       values=[[0.9, 0.3],  # P(Fever=False | Cold)
                               [0.1, 0.7]], # P(Fever=True | Cold)
                       evidence=['Cold'], evidence_card=[2])
# P(RunnyNose | Cold)
cpd_runnynose = TabularCPD(variable='RunnyNose', variable_card=2,
                           values=[[0.8, 0.1],  # P(RunnyNose=False | Cold)
                                   [0.2, 0.9]], # P(RunnyNose=True | Cold)
                           evidence=['Cold'], evidence_card=[2])

# Step 3: Add CPDs to the model
model.add_cpds(cpd_season, cpd_cold, cpd_fever, cpd_runnynose)

# Step 4: Verify the model
print("Is the model valid?", model.check_model())

# Step 5: Perform inference
inference = VariableElimination(model)

# Query 1: What is the probability of having a cold given it's winter (Season=True)?
query_1 = inference.query(variables=['Cold'], evidence={'Season': 1})
print("\nProbability of Cold given Season=True (Winter):")
print(query_1)

# Query 2: What is the probability of having a runny nose given it's winter?
query_2 = inference.query(variables=['RunnyNose'], evidence={'Season': 1})
print("\nProbability of RunnyNose given Season=True (Winter):")
print(query_2)

# Query 3: What is the probability of having a fever and runny nose given a cold?
query_3 = inference.query(variables=['Fever', 'RunnyNose'], evidence={'Cold': 1})
print("\nProbability of Fever and RunnyNose given Cold=True:")
print(query_3)


Is the model valid? True

Probability of Cold given Season=True (Winter):
+---------+-------------+
| Cold    |   phi(Cold) |
+=========+=============+
| Cold(0) |      0.2000 |
+---------+-------------+
| Cold(1) |      0.8000 |
+---------+-------------+

Probability of RunnyNose given Season=True (Winter):
+--------------+------------------+
| RunnyNose    |   phi(RunnyNose) |
+==============+==================+
| RunnyNose(0) |           0.2400 |
+--------------+------------------+
| RunnyNose(1) |           0.7600 |
+--------------+------------------+

Probability of Fever and RunnyNose given Cold=True:
+----------+--------------+------------------------+
| Fever    | RunnyNose    |   phi(Fever,RunnyNose) |
+==========+==============+========================+
| Fever(0) | RunnyNose(0) |                 0.0300 |
+----------+--------------+------------------------+
| Fever(0) | RunnyNose(1) |                 0.2700 |
+----------+--------------+------------------------+
| Fever(1) | R